# Analysis of Census Data and Micromobility Vehicle Locations

In [2]:
import os
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, Normalize, TwoSlopeNorm
from matplotlib.patches import Patch, FancyArrowPatch
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
# file with census data
census_file ='/Users/noamgal/Downloads/ACS_5-Year_Economic_Characteristics_DC_Census_Tract/ACS_5-Year_Economic_Characteristics_DC_Census_Tract.shp'
# file with location data
vehicle_file = '/Users/noamgal/vehicles_20240626_133518.shp'

In [4]:
census_data = gpd.read_file(census_file)

# Process Data

In [6]:
def process_vehicle_files(folder_path, census_file, output_folder):
    """
    Processes multiple vehicle shapefiles, joins each to census tracts,
    and creates individual map visualizations.

    Parameters:
    folder_path (str): Path to the folder containing vehicle shapefiles.
    census_file (str): Path to the census tract shapefile.
    output_folder (str): Path for the output folder to save shapefiles and maps.

    Returns:
    None. Saves output shapefiles and map images for each processed vehicle file.
    """
    # Read the census tract shapefile
    census_data = gpd.read_file(census_file)
    
    # Process each vehicle shapefile
    for filename in os.listdir(folder_path):
        if filename.endswith('.shp') and filename.startswith('vehicles_'):
            vehicle_file = os.path.join(folder_path, filename)
            
            try:
                # Read vehicle data and perform spatial join
                vehicle_data = gpd.read_file(vehicle_file)
                joined_data = gpd.sjoin(vehicle_data, census_data, predicate='within')
                
                # Count vehicles in each census tract
                vehicle_counts = joined_data.groupby('GEOID').size().reset_index(name='veh_count')
                
                # Merge counts back to census tract geometries
                census_data_with_counts = census_data.merge(vehicle_counts, on='GEOID', how='left')
                census_data_with_counts['veh_count'] = census_data_with_counts['veh_count'].fillna(0)
                
                # Generate output filenames
                output_shapefile = os.path.join(output_folder, f"{filename[:-4]}_census_tract_counts.shp")
                output_map = os.path.join(output_folder, f"{filename[:-4]}_census_tract_map.png")
                
                # Export the census data with vehicle counts as a shapefile
                census_data_with_counts.to_file(output_shapefile)
                
                # Create and save the map
                create_map(census_data_with_counts, output_map, filename)
                
                print(f"Processed file: {filename}")
                print(f"Output shapefile saved as: {output_shapefile}")
                print(f"Map saved as: {output_map}")
                print("------------------------")
            
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")

def create_map(data, output_map, source_filename):
    """
    Creates a map visualization of the vehicle counts by census tract.

    Parameters:
    data (GeoDataFrame): The data to be mapped.
    output_map (str): Path for the output map image.
    source_filename (str): Name of the source vehicle file for labeling.

    Returns:
    None. Saves the map as an image file.
    """
    fig, ax = plt.subplots(figsize=(20, 20))

    # Create color map
    vmin = data['veh_count'].min()
    vmax = data['veh_count'].max()
    cmap = plt.cm.YlOrRd

    # Plot the map
    data.plot(column='veh_count', cmap=cmap, linewidth=0.8, edgecolor='0.8', ax=ax)

    # Remove axis
    ax.axis('off')

    # Add title
    plt.title(f"Vehicle Counts by Census Tract in Washington D.C.\nSource: {source_filename}", 
              fontsize=20, fontweight='bold')

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.036, pad=0.04)
    cbar.set_label('Number of Vehicles', fontsize=14)

    # Add labels for each census tract
    for idx, row in data.iterrows():
        centroid = row.geometry.centroid
        ax.annotate(text=f"{int(row['veh_count'])}", 
                    xy=(centroid.x, centroid.y),
                    xytext=(3, 3),
                    textcoords="offset points",
                    fontsize=8,
                    color='black',
                    bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.5))

    # Add north arrow
    x, y, arrow_length = 0.05, 0.95, 0.07
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=48,
                xycoords=ax.transAxes)

    # Add scale bar (simplified version)
    scale_bar_length = 10000  # 10 km in meters
    scale_bar_px = scale_bar_length / (data.total_bounds[2] - data.total_bounds[0]) * fig.get_size_inches()[0] * fig.dpi
    ax.add_patch(plt.Rectangle((0.05, 0.05), scale_bar_px/fig.dpi, 0.005, 
                               facecolor='black', edgecolor='none', transform=ax.transAxes))
    ax.text(0.05, 0.07, '10 km', transform=ax.transAxes, fontsize=12, verticalalignment='bottom')

    # Add source information
    plt.annotate(f'Created on: {pd.Timestamp.now().strftime("%Y-%m-%d")}',
                 xy=(0.05, 0.02), xycoords='figure fraction', fontsize=12, ha='left')

    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_map, dpi=300, bbox_inches='tight')
    plt.close()  # Close the figure to free up memory

# Usage
folder_path = '/Users/noamgal/Downloads/vehicles_final'
census_file = '/Users/noamgal/Downloads/ACS_5-Year_Economic_Characteristics_DC_Census_Tract/ACS_5-Year_Economic_Characteristics_DC_Census_Tract.shp'
output_folder = '/Users/noamgal/Individual-Tract-Maps/'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

process_vehicle_files(folder_path, census_file, output_folder)

Processed file: vehicles_20240626_170044.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_170044_census_tract_counts.shp
Map saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_170044_census_tract_map.png
------------------------
Processed file: vehicles_20240626_150008.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_150008_census_tract_counts.shp
Map saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_150008_census_tract_map.png
------------------------
Processed file: vehicles_20240628_180024.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240628_180024_census_tract_counts.shp
Map saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240628_180024_census_tract_map.png
------------------------
Processed file: vehicles_20240628_200105.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240628_200105_census_tract_counts.shp
Ma

# Average Map Generation

In [ ]:


def process_vehicle_files(folder_path, census_file, output_folder):
    """
    Processes multiple vehicle shapefiles, creates individual and aggregate outputs.

    Parameters:
    folder_path (str): Path to the folder containing vehicle shapefiles.
    census_file (str): Path to the census tract shapefile.
    output_folder (str): Path for the output folder to save results.

    Returns:
    None. Saves output shapefiles and map images.
    """
    # Read the census tract shapefile
    census_data = gpd.read_file(census_file)
    
    all_counts = []
    processed_files = []

    # Process each vehicle shapefile
    for filename in os.listdir(folder_path):
        if filename.endswith('.shp') and filename.startswith('vehicles_'):
            vehicle_file = os.path.join(folder_path, filename)
            
            try:
                # Read vehicle data and perform spatial join
                vehicle_data = gpd.read_file(vehicle_file)
                joined_data = gpd.sjoin(vehicle_data, census_data, predicate='within')
                
                # Count vehicles in each census tract
                vehicle_counts = joined_data.groupby('GEOID').size().reset_index(name='veh_count')
                all_counts.append(vehicle_counts)
                processed_files.append(filename)
                
                # Merge counts back to census tract geometries
                census_data_with_counts = census_data.merge(vehicle_counts, on='GEOID', how='left')
                census_data_with_counts['veh_count'] = census_data_with_counts['veh_count'].fillna(0)
                
                # Generate output filenames for individual file
                output_shapefile = os.path.join(output_folder, f"{filename[:-4]}_census_tract_counts.shp")
                output_map = os.path.join(output_folder, f"{filename[:-4]}_census_tract_map.png")
                
                # Export the census data with vehicle counts as a shapefile
                census_data_with_counts.to_file(output_shapefile)
                
                # Create and save the map for individual file
                create_map(census_data_with_counts, output_map, filename, 'veh_count', 
                           f"Vehicle Counts by Census Tract\nSource: {filename}")
                
                print(f"Processed file: {filename}")
                print(f"Output shapefile saved as: {output_shapefile}")
                print(f"Map saved as: {output_map}")
                print("------------------------")
            
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
    
    if all_counts:
        # Calculate average counts
        avg_counts = pd.concat(all_counts).groupby('GEOID')['veh_count'].mean().reset_index()
        avg_counts['veh_count'] = avg_counts['veh_count'].round(1)
        
        # Merge average counts back to census tract geometries
        census_data_with_avg = census_data.merge(avg_counts, on='GEOID', how='left')
        census_data_with_avg['avg_veh_cnt'] = census_data_with_avg['veh_count'].fillna(0)
        
        # Generate output filenames for aggregate results
        output_avg_shapefile = os.path.join(output_folder, "average_vehicle_counts_census_tract.shp")
        output_avg_map = os.path.join(output_folder, "average_vehicle_counts_census_tract_map.png")
        
        # Export the census data with average vehicle counts as a shapefile
        census_data_with_avg.to_file(output_avg_shapefile)
        
        # Create and save the map for aggregate results
        create_map(census_data_with_avg, output_avg_map, None, 'avg_veh_cnt', 
                   f"Average Vehicle Counts by Census Tract\nBased on {len(processed_files)} observations")
        
        print(f"\nAggregate results:")
        print(f"Output shapefile with average counts saved as: {output_avg_shapefile}")
        print(f"Aggregate map saved as: {output_avg_map}")
        print(f"Number of shapefiles used in the average: {len(processed_files)}")
        print("Files used:")
        for file in processed_files:
            print(f"- {file}")
    else:
        print("No files were successfully processed.")

def create_map(data, output_map, source_filename, count_column, title):
    """
    Creates a map visualization of the vehicle counts by census tract.

    Parameters:
    data (GeoDataFrame): The data to be mapped.
    output_map (str): Path for the output map image.
    source_filename (str): Name of the source vehicle file for labeling (None for aggregate map).
    count_column (str): Name of the column containing the count data.
    title (str): Title for the map.

    Returns:
    None. Saves the map as an image file.
    """
    fig, ax = plt.subplots(figsize=(20, 20))

    # Create color map
    vmin = data[count_column].min()
    vmax = data[count_column].max()
    cmap = plt.cm.YlOrRd
    norm = Normalize(vmin=vmin, vmax=vmax)

    # Plot the map
    data.plot(column=count_column, cmap=cmap, norm=norm, linewidth=0.8, edgecolor='0.8', ax=ax)

    # Remove axis
    ax.axis('off')

    # Add title
    plt.suptitle(title, fontsize=20, fontweight='bold', y=0.95)

    # Add colorbar
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.036, pad=0.04)
    cbar.set_label('Number of Vehicles', fontsize=14)

    # Add labels for each census tract
    for idx, row in data.iterrows():
        centroid = row.geometry.centroid
        ax.annotate(text=f"{row[count_column]:.1f}", 
                    xy=(centroid.x, centroid.y),
                    xytext=(3, 3),
                    textcoords="offset points",
                    fontsize=8,
                    color='black',
                    bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, pad=0.5))

    # Add north arrow
    x, y, arrow_length = 0.05, 0.95, 0.07
    ax.annotate('N', xy=(x, y), xytext=(x, y-arrow_length),
                arrowprops=dict(facecolor='black', width=5, headwidth=15),
                ha='center', va='center', fontsize=48,
                xycoords=ax.transAxes)

    # Add scale bar
    scale_bar_length = 5000  # 5 km in meters
    scale_bar_px = scale_bar_length / (data.total_bounds[2] - data.total_bounds[0]) * fig.get_size_inches()[0] * fig.dpi
    ax.add_patch(plt.Rectangle((0.05, 0.05), scale_bar_px/fig.dpi, 0.005, 
                               facecolor='black', edgecolor='none', transform=ax.transAxes))
    ax.text(0.05, 0.07, '5 km', transform=ax.transAxes, fontsize=12, verticalalignment='bottom')

    # Add source information
    plt.annotate(f'Created on: {pd.Timestamp.now().strftime("%Y-%m-%d")}',
                 xy=(0.05, 0.02), xycoords='figure fraction', fontsize=12, ha='left')

    # Save figure
    plt.tight_layout()
    plt.savefig(output_map, dpi=300, bbox_inches='tight')
    plt.close(fig)  # Close the figure to free up memory

# Usage
folder_path = '/Users/noamgal/Downloads/vehicles_final'
census_file = '/Users/noamgal/Downloads/ACS_5-Year_Economic_Characteristics_DC_Census_Tract/ACS_5-Year_Economic_Characteristics_DC_Census_Tract.shp'
output_folder = '/Users/noamgal/Individual-Tract-Maps/'

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

process_vehicle_files(folder_path, census_file, output_folder)

Processed file: vehicles_20240626_170044.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_170044_census_tract_counts.shp
Map saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_170044_census_tract_map.png
------------------------
Processed file: vehicles_20240626_150008.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_150008_census_tract_counts.shp
Map saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240626_150008_census_tract_map.png
------------------------
Processed file: vehicles_20240628_180024.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240628_180024_census_tract_counts.shp
Map saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240628_180024_census_tract_map.png
------------------------
Processed file: vehicles_20240628_200105.shp
Output shapefile saved as: /Users/noamgal/Individual-Tract-Maps/vehicles_20240628_200105_census_tract_counts.shp
Ma

# Visualizations

In [ ]:
vehicle_file = '/Users/noamgal/Individual-Tract-Maps/average_vehicle_counts_census_tract.shp'
vehicle_data = gpd.read_file(vehicle_file)
print(vehicle_data.columns)
# Export to GeoJSON
vehicle_data.to_file('/Users/noamgal/Downloads/wardvehicles.geojson', driver='GeoJSON')

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots(1, 1, figsize=(20, 15))

# Convert 'DP03_0062E' to numeric, replacing any non-numeric values with NaN
vehicle_data['DP03_0062E'] = pd.to_numeric(vehicle_data['DP03_0062E'], errors='coerce')

# Replace 0 with NaN for median income
vehicle_data.loc[vehicle_data['DP03_0062E'] == 0, 'DP03_0062E'] = np.nan

# Calculate quantiles for color scaling (excluding NaN values)
vmin, vmax = vehicle_data['DP03_0062E'].dropna().quantile([0.05, 0.95])

# Create a custom color map (reversed)
cmap = plt.cm.YlOrRd_r

# Plot the map
vehicle_data.plot(column='DP03_0062E', cmap=cmap, linewidth=0.8, edgecolor='0.8',
                  vmin=vmin, vmax=vmax, ax=ax, missing_kwds={'color': 'lightgrey'})

# Add a title
plt.title('Median Household Income by Census Tract', fontsize=48)

# Remove axis
ax.axis('off')

# Add a color bar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm, cax=cax)
cbar.set_label('Income in $', fontsize=36)
cbar.ax.tick_params(labelsize=24)

# Add north arrow
ax.text(0.95, 0.95, '↑\nN', transform=ax.transAxes, fontsize=60, 
        verticalalignment='top', horizontalalignment='right')

# Add a scale bar
ax.plot([0.05, 0.15], [0.05, 0.05], transform=ax.transAxes, color='k', linewidth=5)
ax.text(0.1, 0.06, '10 km', transform=ax.transAxes, ha='center', va='bottom', fontsize=24)

plt.savefig('map_median_household_income_by_census_tract.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.tight_layout()
plt.show()

# Print information for greyed out tracts
greyed_out_tracts = vehicle_data[vehicle_data['DP03_0062E'].isna()]
print("\nGreyed out census tracts (0 or NaN median income):")
for idx, tract in greyed_out_tracts.iterrows():
    print(f"Name: {tract['NAME20']}, Population: {tract['DP03_0001E']}")

# Print column names
print("\nCensus tract data columns:")
print(vehicle_data.columns)

# Ward Map

In [ ]:
# Load the data
ward_file = '/Users/noamgal/MMgraphs/average_vehicle_counts.shp'
ward_data = gpd.read_file(ward_file)

# Create a figure and axis
fig, ax = plt.subplots(1, 1, figsize=(20, 15))

# Convert 'DP03_0062E' to numeric, replacing any non-numeric values with NaN
ward_data['DP03_0062E'] = pd.to_numeric(ward_data['DP03_0062E'], errors='coerce')

# Remove any rows with NaN values in 'DP03_0062E'
ward_data = ward_data.dropna(subset=['DP03_0062E'])

# Calculate quantiles for color scaling
vmin, vmax = ward_data['DP03_0062E'].quantile([0.05, 0.95])

# Create a custom color map (reversed)
cmap = plt.cm.YlOrRd_r

# Plot the map
ward_data.plot(column='DP03_0062E', cmap=cmap, linewidth=0.8, edgecolor='0.8',
               vmin=vmin, vmax=vmax, ax=ax)

# Add labels for ward names and vehicle count
for idx, row in ward_data.iterrows():
    centroid = row.geometry.centroid
    ax.annotate(f"{row['NAMELSAD']}\nVehicles: {round(row['veh_count']):,}", 
                (centroid.x, centroid.y),
                ha='center', va='center',
                fontsize=20, fontweight='bold',
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='none', pad=1))

# Add a title
plt.title('Median Household Income by Ward', fontsize=48)

# Remove axis
ax.axis('off')

# Add a color bar
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm, cax=cax)
cbar.set_label('Income in $', fontsize=36)
cbar.ax.tick_params(labelsize=24)

# Add north arrow
ax.text(0.95, 0.95, '↑\nN', transform=ax.transAxes, fontsize=60, 
        verticalalignment='top', horizontalalignment='right')

# Add a scale bar
ax.plot([0.05, 0.15], [0.05, 0.05], transform=ax.transAxes, color='k', linewidth=5)
ax.text(0.1, 0.06, '10 km', transform=ax.transAxes, ha='center', va='bottom', fontsize=24)



# Show the plot
plt.tight_layout()
plt.savefig('map_median_household_income_by_ward_with_labels.png', dpi=300, bbox_inches='tight')
plt.show()

# Print column names
print(ward_data.columns)

In [ ]:
def plot_vehicle_counts_and_difference(am_file, pm_file, output_map):
    # Read the shapefiles
    am_data = gpd.read_file(am_file)
    pm_data = gpd.read_file(pm_file)
    
    # Ensure both dataframes have the same index
    am_data.set_index('GEOID', inplace=True)
    pm_data.set_index('GEOID', inplace=True)
    
    # Calculate the difference
    am_data['difference'] = pm_data['veh_count'] - am_data['veh_count']
    
    # Create the map
    fig, ax = plt.subplots(figsize=(40, 36))  # Increased figure size
    
    # Use the RdYlBu diverging colormap
    cmap = plt.get_cmap('RdYlBu')
    
    # Create a custom normalization to handle skewed data
    vmin = am_data['difference'].min()
    vmax = am_data['difference'].max()
    vcenter = 0
    norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
    
    # Plot the difference
    am_data.plot(column='difference', cmap=cmap, norm=norm, linewidth=1.6, edgecolor='0.8', ax=ax)
    
    # Remove axis
    ax.axis('off')
    
    
    # Add colorbar
    sm = ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.08, pad=0.1)
    cbar.set_label('Difference in Vehicle Count (PM - AM)', fontsize=96, labelpad=40)
    cbar.ax.tick_params(labelsize=96)
    
    # Customize colorbar ticks and labels
    tick_locs = [vmin, vmin/2, 0, vmax/2, vmax]
    cbar.set_ticks(tick_locs)
    cbar.set_ticklabels([f'{val:.0f}' for val in tick_locs])
    
    # Add north arrow (more compact)
    north_arrow = FancyArrowPatch((0.05, 0.92), (0.05, 0.97),
                                  transform=ax.transAxes,
                                  color='black',
                                  linewidth=5,
                                  arrowstyle='-|>',
                                  mutation_scale=50)
    ax.add_patch(north_arrow)
    ax.text(0.05, 0.91, 'N', transform=ax.transAxes, fontsize=96, 
            ha='center', va='top', fontweight='bold')

        # Add scale bar
    scale_bar_length = 5000  # 5 km in meters
    scale_bar_px = scale_bar_length / (am_data.total_bounds[2] - am_data.total_bounds[0]) * fig.get_size_inches()[0] * fig.dpi
    ax.add_patch(plt.Rectangle((0.75, 0.05), scale_bar_px/fig.dpi, 0.01, 
                               facecolor='black', edgecolor='none', transform=ax.transAxes))
    ax.text(0.75, 0.08, '5 km', transform=ax.transAxes, fontsize=48, verticalalignment='bottom')

    
    # Adjust layout and save
    plt.tight_layout()
    plt.savefig(output_map, dpi=300, bbox_inches='tight')
    print(f"Map saved as: {output_map}")
    
    # Display the map
    plt.show()
    
    # Print explanation of color interpretation
    print("\nColor Interpretation (2-Slope Normalized):")
    print(f"Red areas: More vehicles in AM (up to {vmin:.0f} fewer vehicles in PM)")
    print("White areas: Little to no difference between AM and PM vehicle counts")
    print(f"Blue areas: More vehicles in PM (up to {vmax:.0f} more vehicles in PM)")
    print("\nTwo-slope Normalization Explanation:")
    print("The color scale is normalized separately for positive and negative values.")
    print("This allows for better distinction of differences on both sides of zero,")
    print("even when the range of values is not symmetric around zero.")
    # Print the range of differences
    print(f"\nRange of differences: {vmin:.0f} to {vmax:.0f}")
    print("Negative values indicate more vehicles in AM, positive values indicate more vehicles in PM")

# Usage
am_vehicles_file = '/Users/noamgal/Individual-Tract-Maps/vehicles_20240628_150019_census_tract_counts.shp'
pm_vehicles_file = '/Users/noamgal/Individual-Tract-Maps/vehicles_20240628_210025_census_tract_counts.shp'
output_map = '/Users/noamgal/Individual-Tract-Maps/vehicle_count_difference_pm_am_census_tracts.png'

plot_vehicle_counts_and_difference(am_vehicles_file, pm_vehicles_file, output_map)